Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 2 
Andres Quinto - 18288  

## TASK 1 - Preguntas teóricas

1. ¿Por qué el modelo de Naive Bayes se le considera "naive"?  

    - Se le considera "naive" debido a que el algoritmo asume que la presencia o ausencia de una característica no está influenciada por la presencia o ausencia de otras características en el conjunto de datos. En el problema de SPAM & HAM podemos verlo reflejado en que trata a todas las palabras de igual forma no importa el orden en el que esten en una oracion, esto, debido a que Naive Bayes trata al lenguaje como una bolsa de palabras y cada mensaje siendo una parte random de lo que esta dentro de la bolsa. (Informacion sacada del video de naive bayes del modulo de esta semana)
    [(Naive Bayes, Clearly Explained, 2021)](https://www.youtube.com/watch?v=O2L2Uv9pdDA)   
    
2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase) 
   
    - Lo que trata SVM es mover la data a espacios con mayor dimensiones. Con ello obtenemos dos cosas importantes que son lo que se busca optimizar, un hiperplano y un margen. SVM intenta encontrar un hiperplano que separe de manera óptima las dos clases. El objetivo de la formulación se centra en maximizar el margen entre las clases.  
    
    La definicion del plano, que en este caso es la linea divisoria que nos ayuda a clasificar esta dada por:  
    $$ w^Tx + b = 0  
    $$
    Donde:
    1. w es el vector de pesos.
    2. x es el vector de características.
    3. b es el sesgo.

    La condición de clasificación para un punto de datos es:

    $$ y_i = \text{sign}(w^Tx_i + b)$$

    El objetivo es maximizar el margen, sin embargo, para facilitar la optimización, se minimiza  $$ 1/2 ||w||^2$$ sujeto a restricciones que todos los puntos de datos se clasifique correctamente:
    $$ y_i(w^Tx_i + b) \geq 1, \quad \text{para todo } i$$

    Formulación del Problema de Optimización con Multiplicadores de Lagrange  
    
    - Multiplicadores de Lagrange: Se utilizan para incorporar estas restricciones en la función objetivo.  
    - Función Lagrangiana: Combina la función objetivo y las restricciones, cada una ponderada por un multiplicador de Lagrange.  
    - Minimizar la Lagrangiana: Encontramos: $$ w, b, α $$(multiplicadores de Lagrange) que minimicen la función Lagrangiana.  
    
    Introduciendo los multiplicadores de Lagrange $$( \alpha_i \geq 0 )$$ la función Lagrangiana $$( L ) $$ es:  
    $$[ L(w, b, \alpha) = \frac{1}{2}\|w\|^2 - \sum_{i=1}^n \alpha_i [y_i(w^Tx_i + b) - 1] ]$$  

    Esto es util por que convierte todo nuestro problema en uno de optimización estándar (programación cuadrática), ademas de transformar un problema de optimización con restricciones en uno manejable, permitiendo soluciones óptimas eficientes y aplicables a problemas lineales y no lineales.  
    
    [(Rogers, S., & Girolami, M, 2016)](https://github.com/wwkenwong/book/blob/master/Simon%20Rogers%2C%20Mark%20Girolami%20A%20First%20Course%20in%20Machine%20Learning.pdf)  

3. Investigue sobre Random Forest y responda:  

    a. ¿Qué tipo de ensemble learning es este modelo?  
        Random Forest es un método de aprendizaje ensamble (ensemble learning), también conocido como aprendizaje por agregación (bagging), que combina la salida de múltiples modelos de aprendizaje automático para mejorar la precisión y la robustez del modelo final. En el caso de Random Forest, los modelos individuales son árboles de decisión.  

    b. ¿Cuál es la idea general detrás de Random Forest?  
        Conforme la explicación del video del modulo [(¿Qué es Decision Tree y Random Forest?, 2021)](https://www.youtube.com/watch?v=tYPi6qcCQbg&t=1s).  
        Utilizaban la analogia de que el random forest era como una democracia, donde cada uno de los datasets que se generaban eran el resultado luego de su democracia y a lo macro eran varios forest. La idea central de Random Forest es combinar múltiples árboles de decisión para obtener un modelo más robusto y preciso.  Este proceso aumenta la diversidad entre los árboles, lo que ayuda a mejorar el rendimiento general y reduce el riesgo de sobreajuste, común en los árboles de decisión individuales.  

    c. ¿Por qué se busca baja correlación entre los árboles de Random Forest?  
        La baja correlación entre los árboles de Random Forest ayuda a reducir el sesgo y el sobreajuste. Si los árboles de decisión están altamente correlacionados, es probable que aprendan los mismos patrones y, por lo tanto, sean más propensos al sesgo y al sobreajuste.  
[(Random Forests, 2001)](https://link.springer.com/article/10.1023/A:1010933404324)  
[(¿Qué es Decision Tree y Random Forest?, 2021)](https://www.youtube.com/watch?v=tYPi6qcCQbg&t=1s)   

## Task 2 - Naive Bayes: Clasificador de Mensajes Ham/Spam

### TASK 2.1

In [37]:
## LIMPIEZA DEL DATASET
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict
from math import log

def clean_dataset():
    """
    Cleans the dataset by removing special characters, converting text to lowercase,
    splitting the dataset by newline character, and processing each observation.
    Returns the processed observations.
    """
    # Read the dataset file
    with open("entrenamiento.txt", "r") as file:
        dataset = file.read()

    # Remove special characters
    clean_dataset = re.sub(r"[^a-zA-Z0-9\s]", "", dataset)

    # Convert text to lowercase
    clean_dataset = clean_dataset.lower()

    # Split the cleaned dataset by newline character
    observations = clean_dataset.split("\n")

    # Process each observation
    processed_observations = []
    for observation in observations:
        # Split the observation by tab character
        parts = observation.split("\t")
        if len(parts) >= 2:
            label = parts[0]  # Extract the label
            message = "\t".join(parts[1:])  # Extract the message
            processed_observations.append((label, message))

    return processed_observations

# # Print the processed observations in the format specified Label and Message
# for label, message in clean_dataset():
#     print(f"Label: {label}\tMessage: {message}")

# Divide el dataset en conjuntos de entrenamiento y prueba en una proporcion de 20 para test y 80 para entrenamiento
train_set, test_set = train_test_split(clean_dataset(), test_size=0.2, train_size=0.8, random_state=42)


###BAYES NAIVE MODEL

def train_naive_bayes(train_set):
    """
    Trains a Naive Bayes model with Laplace smoothing using the training set.
    Returns the trained model.
    """
    # Count the occurrences of each word in spam and ham messages
    word_counts_spam = defaultdict(int)
    word_counts_ham = defaultdict(int)
    total_words_spam = 0
    total_words_ham = 0

    for label, message in train_set:
        words = re.findall(r'\w+', message)  # Extract words from the message
        if label == 'spam':
            for word in words:
                word_counts_spam[word] += 1
                total_words_spam += 1
        else:
            for word in words:
                word_counts_ham[word] += 1
                total_words_ham += 1

    # Calculate the probabilities of each word occurring in spam and ham messages
    word_probs_spam = defaultdict(float)
    word_probs_ham = defaultdict(float)

    for word, count in word_counts_spam.items():
        word_probs_spam[word] = (count + 1) / (total_words_spam + len(word_counts_spam))

    for word, count in word_counts_ham.items():
        word_probs_ham[word] = (count + 1) / (total_words_ham + len(word_counts_ham))

    # Calculate the prior probabilities of spam and ham
    total_messages = len(train_set)
    spam_messages = sum(1 for label, _ in train_set if label == 'spam')
    ham_messages = total_messages - spam_messages
    prior_spam = spam_messages / total_messages
    prior_ham = ham_messages / total_messages
    # Calculate vocabulary size (unique words in both spam and ham)
    vocab_size = len(set(word_counts_spam.keys()).union(set(word_counts_ham.keys())))

    # Return the trained model
    return word_probs_spam, word_probs_ham, prior_spam, prior_ham, vocab_size, total_words_ham, total_words_spam

def classify_message(message, word_probs_spam, word_probs_ham, prior_spam, prior_ham, vocab_size, total_words_ham, total_words_spam):
    """
    Classifies a message as spam or ham using the trained Naive Bayes model.
    Returns the predicted label.
    """
    words = re.findall(r'\w+', message)

    log_likelihood_spam = 0
    log_likelihood_ham = 0

    for word in words:
        # Using Laplace smoothing for unseen words
        prob_spam = word_probs_spam.get(word, 1 / (total_words_spam + vocab_size))
        prob_ham = word_probs_ham.get(word, 1 / (total_words_ham + vocab_size))

        log_likelihood_spam += log(prob_spam)
        log_likelihood_ham += log(prob_ham)

    log_posterior_spam = log_likelihood_spam + log(prior_spam)
    log_posterior_ham = log_likelihood_ham + log(prior_ham)

    return 'spam' if log_posterior_spam > log_posterior_ham else 'ham'


# Train the Naive Bayes model
word_probs_spam, word_probs_ham, prior_spam, prior_ham, vocab_size, total_words_spam, total_words_ham = train_naive_bayes(train_set)

# Classify test messages using the trained model
for label, message in test_set:
    predicted_label = classify_message(
        message, word_probs_spam, word_probs_ham, prior_spam, prior_ham, vocab_size, total_words_ham, total_words_spam
    )
    print(f"Message: {message}\nTrue Label: {label}\tPredicted Label: {predicted_label}\n")


Message: how many licks does it take to get to the center of a tootsie pop
True Label: ham	Predicted Label: ham

Message: your weekly coolmob tones are ready to download this weeks new tones include 1 crazy frogaxel f 2 akonlonely 3 black eyeddont p more info in n
True Label: spam	Predicted Label: spam

Message: eastenders tv quiz what flower does dot compare herself to d violet e tulip f lily txt d e or f to 84025 now 4 chance 2 win 100 cash wkent150p16
True Label: spam	Predicted Label: spam

Message: this phone has the weirdest auto correct
True Label: ham	Predicted Label: ham

Message: if you mean the website yes
True Label: ham	Predicted Label: ham

Message: no thank you youve been wonderful
True Label: ham	Predicted Label: ham

Message: ok i found dis pierre cardin one which looks normal costs 20 its on sale
True Label: ham	Predicted Label: ham

Message: somewhere out there beneath the pale moon light someone think in of u some where out there where dreams come true goodnite amp s

## Task 2.2 - Construcción del modelo
